<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Profile_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install


In [15]:
# Enter url, email and password
user_url = "https://www.linkedin.com/in/daniel-behar-168647280/"
email = "daniel10behar@gmail.com"
password = "PASS"


In [13]:
import nest_asyncio
from playwright.async_api import async_playwright
import asyncio

async def scrape_user_profile(user_url, email, password):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Running in headless mode
        page = await browser.new_page()

        print("Navigating to LinkedIn login page...")
        # Navigate to LinkedIn login page
        await page.goto("https://www.linkedin.com/login")
        await asyncio.sleep(3)

        # Perform login
        print("Logging into LinkedIn...")
        await page.fill('input[id="username"]', email)
        await page.fill('input[id="password"]', password)
        await page.click('button[type="submit"]')
        await asyncio.sleep(5)

        # Check for two-step verification
        if await page.is_visible('input[name="pin"]'):
            print("Two-step verification detected. Please enter the code sent to your email.")
            verification_code = input("Enter the verification code: ")
            await page.fill('input[name="pin"]', verification_code)
            await page.click('button[type="submit"]')
            await asyncio.sleep(3)

        # Relevant sections to scrape for the user
        profile_sections = [
            "/details/skills/",
            "/details/interests/",
            "/details/recommendations/",
            "/details/certifications/"
        ]

        scraped_data = {}  # Dictionary to store HTML content of each section

        # Loop through the list of profile URLs
        for section in profile_sections:
            url = f"{user_url}{section}"
            try:
                print(f"Navigating to section: {url}")
                # Navigate to the section URL
                await page.goto(url)
                await asyncio.sleep(5)

                # Save the HTML content of the section page
                html = await page.content()
                section_name = section.strip("/").split("/")[-1]  # Keep only the part after the last slash
                scraped_data[section_name] = html  # Store HTML content in dictionary
                print(f"HTML content for {section_name} saved in memory.")

                # For debug
                filename = f"{section_name}.html"  # Ensure .html extension
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(html)
                print(f"Saved HTML for {url} as {section_name}")

                # Optional: Take a screenshot of the section page
                screenshot_filename = f"{section_name}.png"
                await page.screenshot(path=screenshot_filename, full_page=True)
                print(f"Saved screenshot for {url} as {screenshot_filename}")

            except Exception as e:
                print(f"Failed to scrape {url}: {e}")

        # Close the browser
        print("Closing the browser...")
        await browser.close()

        return scraped_data  # Return all scraped HTML as a dictionary



In [16]:
scraped_html_data = await scrape_user_profile(user_url, email, password)

from bs4 import BeautifulSoup

# Example HTML from the dictionary
html_content = scraped_html_data.get("skills")

# Parse HTML
soup = BeautifulSoup(html_content, "html.parser")



Navigating to LinkedIn login page...
Logging into LinkedIn...
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280//details/skills/
HTML content for skills saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280//details/skills/ as skills
Saved screenshot for https://www.linkedin.com/in/daniel-behar-168647280//details/skills/ as skills.png
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280//details/interests/
HTML content for interests saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280//details/interests/ as interests
Saved screenshot for https://www.linkedin.com/in/daniel-behar-168647280//details/interests/ as interests.png
Navigating to section: https://www.linkedin.com/in/daniel-behar-168647280//details/recommendations/
HTML content for recommendations saved in memory.
Saved HTML for https://www.linkedin.com/in/daniel-behar-168647280//details/recommendations/ as recommendations
Sav

In [17]:
# Find all <a> tags with data-field="skill_page_skill_topic"
skill_links = soup.find_all("a", {"data-field": "skill_page_skill_topic"})

# Extract skills
skills = []
for link in skill_links:
    # Find the <span> with aria-hidden="true" inside the link
    skill_span = link.find("span", {"aria-hidden": "true"})
    if skill_span:
        skill_text = skill_span.get_text(strip=True)
        skills.append(skill_text)

unique_skills = sorted(set(skills))
print("Unique skills:", unique_skills)


Unique skills: ['Apache Spark', 'Data Analysis', 'Data Structures', 'Deep Learning', 'Django', 'HTML', 'Java', 'PyTorch', 'Python (Programming Language)', 'SQL', 'Statistical Data Analysis']
